In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %matplotlib widget

import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import wandb

import neural_network_lyapunov.examples.quadrotor3d.quadrotor as quadrotor
import neural_network_lyapunov.utils as utils
import neural_network_lyapunov.feedback_system as feedback_system
import neural_network_lyapunov.lyapunov as lyapunov
import neural_network_lyapunov.train_lyapunov as train_lyapunov

In [ ]:
dtype = torch.float64
forward_model = torch.load('models/forward_model.pt')
forward_model

In [ ]:
dt = 1./5.
hover_thrust = .705
xu_equ = torch.tensor([0., 0., 0., 0., 0., 0., hover_thrust], dtype=dtype)
x_lo = torch.tensor([-.25, -.25, -.25, -np.pi/4, -np.pi/4, -np.pi/4, -5, -5, -5], dtype=dtype)
x_up = torch.tensor([.25, .25, .25, np.pi/4, np.pi/4, np.pi/4, 5, 5, 5], dtype=dtype)
u_lo = torch.tensor([-np.pi/10, -np.pi/10, -np.pi/10, 0.70], dtype=dtype)
u_up = torch.tensor([np.pi/10, np.pi/10, np.pi/10, 0.71], dtype=dtype)
forward_system = quadrotor.QuadrotorWithPixhawkReLUSystem(
    dtype, x_lo, x_up, u_lo, u_up, forward_model, hover_thrust, dt)

In [ ]:
u_traj = torch.zeros((300, 4)).double()
u_traj[:, 3] = .71 * torch.ones(u_traj.shape[0])

mag = np.pi/1000

u_traj1 = u_traj.clone()
u_traj1[:, 0] = mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

u_traj2 = u_traj.clone()
u_traj2[:, 0] = -mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

u_traj3 = u_traj.clone()
u_traj3[:, 1] = mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

u_traj4 = u_traj.clone()
u_traj4[:, 1] = -mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

In [ ]:
xx_traj = []
for u_traj in [u_traj1, u_traj2, u_traj3, u_traj4]:
    x_traj = [torch.tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=dtype)]
    for n in range(u_traj.shape[0]):
        x_traj.append(forward_system.step_forward(x_traj[-1], u_traj[n, :]))
    x_traj = torch.cat([x.unsqueeze(0).detach() for x in x_traj])
    xx_traj.append(x_traj)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for x_traj in xx_traj:
    plt.plot(x_traj[:, 0], x_traj[:, 1], 'b')
    plt.plot(x_traj[0, 0], x_traj[0, 1], 'g.')
    plt.plot(x_traj[-1, 0], x_traj[-1, 1], 'r*')
ax.set_aspect('equal')
plt.show()

In [ ]:
u_traj = torch.zeros((300, 4)).double()
u_traj[:, 3] = .72 * torch.ones(u_traj.shape[0])

mag = np.pi/1000

u_traj1 = u_traj.clone()
u_traj1[:, 0] = mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))
u_traj1[:, 1] = mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

u_traj2 = u_traj.clone()
u_traj2[:, 0] = -mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))
u_traj2[:, 1] = -mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

u_traj3 = u_traj.clone()
u_traj3[:, 0] = -mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))
u_traj3[:, 1] = mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

u_traj4 = u_traj.clone()
u_traj4[:, 0] = mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))
u_traj4[:, 1] = -mag * torch.sin(torch.linspace(0, np.pi, u_traj.shape[0]))

In [ ]:
xx_traj = []
for u_traj in [u_traj1, u_traj2, u_traj3, u_traj4]:
    x_traj = [torch.tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=dtype)]
    for n in range(u_traj.shape[0]):
        x_traj.append(forward_system.step_forward(x_traj[-1], u_traj[n, :]))
    x_traj = torch.cat([x.unsqueeze(0).detach() for x in x_traj])
    xx_traj.append(x_traj)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for x_traj in xx_traj:
    plt.plot(x_traj[:, 0], x_traj[:, 1], 'b')
    plt.plot(x_traj[0, 0], x_traj[0, 1], 'g.')
    plt.plot(x_traj[-1, 0], x_traj[-1, 1], 'r*')
ax.set_aspect('equal')
plt.show()

In [ ]:
controller_model = torch.load('models/controller.pt')
x_equilibrium = torch.tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=dtype)
u_equilibrium = torch.tensor([0., 0., 0., .705], dtype=dtype)
u_lo = torch.tensor([-np.pi/20, -np.pi/20, -np.pi/4, 0.70], dtype=dtype)
u_up = torch.tensor([np.pi/20, np.pi/20, np.pi/4, 0.72], dtype=dtype)
        
def get_control(x_current):
    u_current = (controller_model(x_current) - 
        controller_model(x_equilibrium) + u_equilibrium)
    u_current = torch.min(torch.max(u_current, u_lo), u_up)
    return u_current

x_traj = [torch.tensor([0.01, 0.01, 0., 0., 0., 0., 0., 0., 0.], dtype=dtype)]
for n in range(u_traj.shape[0]):
    u = get_control(x_traj[-1])
    x_traj.append(forward_system.step_forward(x_traj[-1], u))
x_traj = torch.cat([x.unsqueeze(0).detach() for x in x_traj])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(x_traj[:, 0], x_traj[:, 1], 'b')
plt.plot(x_traj[0, 0], x_traj[0, 1], 'g.')
plt.plot(x_traj[-1, 0], x_traj[-1, 1], 'r*')
ax.set_aspect('equal')
plt.show()